<a href="https://colab.research.google.com/github/AlfredoLSN/Limpeza-de-Dados/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Base de dados

In [515]:
import pandas as pd
import plotly.express as px

In [516]:
base = pd.read_excel("/content/drive/MyDrive/ChavesClientes.xlsx")
base.head(10)

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0
5,6,28FC,C-Amarelo,Platinum-Alpha,0
6,7,27MD,A-Verde,Platinum-Beta,1
7,8,31MD,C-Cinza,Basic,0
8,9,28FS,A-Cinza,Black,1
9,10,31MV,C+Amarelo,Platinum,1


### Usando a base de dados dessa forma não conseguimos fazer associações entre as colunas, como é mostrado nos graficos a seguir:

In [517]:
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

In [518]:
px.treemap(base, path = ["ChaveSituacao", "Pagamento"], title = "Agrupamento entre ChaveSituacao e Pagamento")

In [519]:
px.treemap(base, path = ["ClassRisco", "Pagamento"], title = "Agrupamento entre ClassRisco e Pagamento")

In [520]:
px.treemap(base, path = ["CatCliente", "Pagamento"], title = "Agrupamento entre CatCliente e Pagamento")

## Informações da base de dados

- Caracteristicas dos dados:
    - ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
    - ClassRisco: formado por:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa
    - CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
        - Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta

## Limpeza dos Dados

### Irei tratar a coluna ChaveSituação, onde apartir dela irei criar tres novas colunas, Idade, Genero e EstadoCivil

In [521]:
base["Idade"] = base["ChaveSituacao"].str[0:2]
base["Genero"] = base["ChaveSituacao"].str[2]
base["EstadoCivil"] = base["ChaveSituacao"].str[3]
base["Idade"] = pd.to_numeric(base["Idade"])
base = base.drop("ChaveSituacao", axis = 1)
base.head(10)

,ID,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,Ccinza,Basic-Alpha,1,32,F,C
1,2,AAmarelo,Black,1,25,M,V
2,3,B-Amarelo,Basic-Beta,1,27,M,V
3,4,BAmarelo,Black,0,26,F,D
4,5,C-Amarelo,Black,0,26,F,D
5,6,C-Amarelo,Platinum-Alpha,0,28,F,C
6,7,A-Verde,Platinum-Beta,1,27,M,D
7,8,C-Cinza,Basic,0,31,M,D
8,9,A-Cinza,Black,1,28,F,S
9,10,C+Amarelo,Platinum,1,31,M,V


### Agora irei tratar a coluna CatCliente, apartir dela irei criar as colunas CategoriaCartao e Categoria VIP

In [522]:
base["CategoriaCartao"] = base["CatCliente"].str.split("-").str.get(0)
base["CategoriaVip"] = base["CatCliente"].str.split("-").str.get(1)
base.head(10)

,ID,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,CategoriaCartao,CategoriaVip
0,1,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,C-Amarelo,Black,0,26,F,D,Black,NaN
5,6,C-Amarelo,Platinum-Alpha,0,28,F,C,Platinum,Alpha
6,7,A-Verde,Platinum-Beta,1,27,M,D,Platinum,Beta
7,8,C-Cinza,Basic,0,31,M,D,Basic,NaN
8,9,A-Cinza,Black,1,28,F,S,Black,NaN
9,10,C+Amarelo,Platinum,1,31,M,V,Platinum,NaN


####Muitos registros na coluna CatCliente nao possuiam o campo CategoriaVip, logo a coluna CategoriaVip ficou com bastante dados NaN, ou seja, dados que não foram informados.Para resolver esse problema irei substituir todos os NaN por Comum, supondo que os usuarios que nao possuem o campo Vip são usuario comuns.

In [523]:
base["CategoriaVip"].isnull().sum() #Contar qauntos registros possuem o campo Vip como NaN

12

In [524]:
base.loc[base["CategoriaVip"].isnull(), "CategoriaVip"] = "Comum"
base = base.drop("CatCliente", axis = 1)
base.head(10)

,ID,ClassRisco,Pagamento,Idade,Genero,EstadoCivil,CategoriaCartao,CategoriaVip
0,1,Ccinza,1,32,F,C,Basic,Alpha
1,2,AAmarelo,1,25,M,V,Black,Comum
2,3,B-Amarelo,1,27,M,V,Basic,Beta
3,4,BAmarelo,0,26,F,D,Black,Comum
4,5,C-Amarelo,0,26,F,D,Black,Comum
5,6,C-Amarelo,0,28,F,C,Platinum,Alpha
6,7,A-Verde,1,27,M,D,Platinum,Beta
7,8,C-Cinza,0,31,M,D,Basic,Comum
8,9,A-Cinza,1,28,F,S,Black,Comum
9,10,C+Amarelo,1,31,M,V,Platinum,Comum


### Agora irei tratar a coluna ClassRisco, apartir dela irei criar as colunas Classificação e Cor

In [525]:
import re

####Regex para pegar a primeira Letra e o simbolo seguinte caso exista

In [526]:
re.findall("^[A-Z]\W?", "B+teste"),re.findall("^[A-Z]\W?", "B-teste"), re.findall("^[A-Z]\W?", "Bteste"), re.findall("^[A-Z]\W?", "BBteste"), re.findall("^[A-Z]\W?", "Bbteste")

(['B+'], ['B-'], ['B'], ['B'], ['B'])

In [527]:
base["Classificacao"] = base["ClassRisco"].apply(lambda x: re.findall("^[A-Z]\W?", x)[0])
base["Cor"] = base["ClassRisco"].apply(lambda x: re.sub("^[A-Z]\W?", "", x))
base = base.drop("ClassRisco", axis = 1)
base.head(10)

,ID,Pagamento,Idade,Genero,EstadoCivil,CategoriaCartao,CategoriaVip,Classificacao,Cor
0,1,1,32,F,C,Basic,Alpha,C,cinza
1,2,1,25,M,V,Black,Comum,A,Amarelo
2,3,1,27,M,V,Basic,Beta,B-,Amarelo
3,4,0,26,F,D,Black,Comum,B,Amarelo
4,5,0,26,F,D,Black,Comum,C-,Amarelo
5,6,0,28,F,C,Platinum,Alpha,C-,Amarelo
6,7,1,27,M,D,Platinum,Beta,A-,Verde
7,8,0,31,M,D,Basic,Comum,C-,Cinza
8,9,1,28,F,S,Black,Comum,A-,Cinza
9,10,1,31,M,V,Platinum,Comum,C+,Amarelo


###Agora que a base de dados esta mais limpa podemos ver melhor as correlações entre as colunas

In [528]:
px.treemap(base, path = ["Genero", "Pagamento"], title = "Agrupamento entre Genero e Pagamento")

In [529]:
px.treemap(base, path = ["CategoriaCartao", "Pagamento"], title = "Agrupamento entre CategoriaCartao e Pagamento")

In [530]:
px.treemap(base, path = ["CategoriaVip", "Pagamento"], title = "Agrupamento entre CategoriaVip e Pagamento")

In [531]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               20 non-null     int64 
 1   Pagamento        20 non-null     int64 
 2   Idade            20 non-null     int64 
 3   Genero           20 non-null     object
 4   EstadoCivil      20 non-null     object
 5   CategoriaCartao  20 non-null     object
 6   CategoriaVip     20 non-null     object
 7   Classificacao    20 non-null     object
 8   Cor              20 non-null     object
dtypes: int64(3), object(6)
memory usage: 1.5+ KB
